# GC Fisher Matrix Code

Authors: Alkistis Pourtsidou and Dida Markovic, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

## Initialise parameters and import libraries

In [ ]:
%matplotlib inline

# Set up all kinds of libraries
import sys, platform
from matplotlib import pyplot as plt
import os

# Set up science stuff
import numpy as np
import scipy
from scipy.interpolate import interp1d
from __future__ import division
from scipy import integrate, linalg

# Import custom modules in this folder
import misc

# Importing CAMB
import camb
from camb import model, initialpower

In [ ]:
# Options for running this notebook

### Flags and paths
interp_type = 'linear'    # Type of interpolation
outpath = './outputs/'    # Where to save outputs
TS = misc.get_timestamp() # Arbitrary time stamp so linked output files are obvious

### Fiducial cosmological parameters
hubble_fid = 0.67
omegab_fid = 0.022445/hubble_fid**2
omegac_fid = 0.121203/hubble_fid**2
om0_fid = omegac_fid + omegab_fid
H00_fid = 100*hubble_fid
Ass_fid = 2.1265e-9
nss_fid = 0.96
gamma_fid = 0.545
c = 3e5

### Set up the survey
Area = 15000.0 #deg^2
omegatot = Area*pow(np.pi/180,2)
Dzbin = 0.1
sig_p = 0.001

### Set up the redshift binned functions from IST docs
zlist = np.arange(0.7,2.1,Dzbin)
Nzbins = len(zlist)

biaslist = [1.083, 1.125, 1.104, 1.126, 1.208, 1.243, 1.282, 1.292, 1.363, 1.497, 1.486, \
            1.491, 1.573, 1.568]

kmin = 0.001
kmax = 0.2

dn3 = [2434.28, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, \
       2308.975, 1541.831, 1474.707, 893.716, 497.613]

### Set up the derivative steps
z_dep_step_default = 1e-5
shape_step = 1e-3

### Set up the Fisher matrix calculation
params = {
    0: 'lnH',
    1: 'lnDA',
    2: 'lnfsig8',
    3: 'lnbsig8',
    4: 'Ps',
    5: 'ns',
    6: 'wb',
    7: 'wm',
    8: 'h'}
shape_params = [5,6,7,8]

## Define functions

In [ ]:
# Geometry functions for a spatially flat Universe

# Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0_fid+om0_fid*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00_fid

# Define the cosmological distances
def drdz(zp):
    return (c/H00_fid)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)


In [ ]:
# LCDM growth rate and growth factor
def fg(zz):
    omz=om0_fid*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma_fid)

def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    start_z = 0.0
    ans = scipy.integrate.romberg(Dg_dz, start_z, zc)
    return np.exp(ans)
    
print fg(0.7), fg(1.7)

In [ ]:
# Get the power spectrum from CAMB
nofk = 800

# Set up the fiducial cosmology for CAMB
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00_fid, 
                   ombh2=omegab_fid*pow(hubble_fid,2), 
                   omch2=omegac_fid*pow(hubble_fid,2),
                   omk=0,
                   mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss_fid, r=0, As=Ass_fid)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

# Calculate results for these parameters
results_fid = camb.get_results(pars)

# Get linear matter power spectrum at z=0: P(k,z=0)
pars.set_matter_power(redshifts=[0.], kmax=2.0)
pars.NonLinear = model.NonLinear_none
results_fid.calc_power_spectra(pars)    
kh, _z, pk = results_fid.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints=nofk)

sig8_fid = results_fid.get_sigma8()[0]
print sig8_fid # 0.830287480109

rt0_fid = interp1d(kh, pk[0]/pow(sig8_fid,2), kind=interp_type)
print rt0_fid(0.1) # 8274.87712618

Pkz0 = interp1d(kh, pk[0], kind=interp_type)
print Pkz0(0.1) # 5704.51244798

def reset_cosmology(h=hubble_fid, 
                    wb=omegab_fid*pow(hubble_fid,2), 
                    wm=(omegac_fid+omegab_fid)*pow(hubble_fid,2), 
                    ns=nss_fid):

    # Reset CAMB
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=100.0*h, ombh2=wb, omch2=wm-wb,omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=ns, r=0, As=Ass_fid)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);

    # Calculate results for these parameters
    results = camb.get_results(pars)

    # Get linear matter power spectrum at z=0: P(k,z=0)
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

    kh, _z, pk = results.get_matter_power_spectrum(minkh=1e-4,maxkh=1.0,npoints=nofk)
    sigma8 = results.get_sigma8()[0]
    ratio = interp1d(kh, pk[0]/pow(sigma8,2), kind=interp_type)

    return results, ratio

# Provides this function in the fromfile = False case.
# z-dependence goes away in this implementation.
def sig8(zc=0.0, results=results_fid):
    return results.get_sigma8()[0]

# Provides the P(k,z) funcion in the fromfile = False case.
def Pkz(kk,zc,ratio=rt0_fid,results=results_fid):
    return pow(Dgz(zc),2)*ratio(kk)*sig8(results=results)**2

In [ ]:
# Construct the observed power spectrum, P_gg(k,μ,z)
def Pgg(kk,mu,zc,bgs8=None,fgs8=None):
    s8 = sig8(zc)
    if bgs8 is None:
        bgs8 = bg * s8
    if fgs8 is None:
        fgs8 = fg(zc) * s8
    return pow(bgs8+fgs8*mu**2,2)*Pkz(kk,zc)/s8**2

print Pkz(0.1,0.8)
print Pgg(0.1,0.,0.8,bgs8=sig8(0.8))

In [ ]:
# Calculate the survey properties

# Redshift errors

def photoz(kk,mu,zc):
    sigz = sig_p*(1+zc) 
    return np.exp(-pow(kk*mu,2)*pow(c*sigz,2)/pow(Hz(zc),2))
#print photoz(0.5,0.5,ztest)

# Survey (bin) volume [Mpc^3]

def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00_fid*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

def Pshot(zc):
    return Vsur(zc)/Ngal

# Effective volume (the hubble**3 factors are for units consistency)
def Veff(kk,mu,zc):
    return hubble_fid**3*Vsur(zc)* \
           (Pgg(kk,mu,zc)*photoz(kk,mu,zc)/(Pgg(kk,mu,zc)*photoz(kk,mu,zc) \
           + hubble_fid**3*Pshot(zc)))**2

In [ ]:
# Fisher matrix derivatives 

#the σ8's cancel out so we don't include them

# Shot noise derivative analytically since it is trivial
def dlnP_dPs(kk,mu,zc):
    return 1.0/(Pgg(kk,mu,zc)*photoz(kk,mu,zc))

# NUMERICAL DERIVATIVES
def dlnP_dlnfsig8(kk,mu,zc,step_fs8=z_dep_step_default):
    fs8_fid = fg(zc)*sig8(zc)
    fs8_p = fs8_fid*(1+step_fs8)
    fs8_m = fs8_fid*(1-step_fs8)

    Pgg_fid = Pgg(kk,mu,zc)
    Pgg_p = Pgg(kk,mu,zc,fgs8=fs8_p)
    Pgg_m = Pgg(kk,mu,zc,fgs8=fs8_m)

    return (fs8_fid/Pgg_fid)*(Pgg_p-Pgg_m)/(2*step_fs8*fs8_fid)

def dlnP_dlnbsig8(kk,mu,zc,step_bs8=z_dep_step_default):
    bs8_fid = bg*sig8(zc)
    bs8_p = bs8_fid*(1+step_bs8)
    bs8_m = bs8_fid*(1-step_bs8)

    Pgg_fid = Pgg(kk,mu,zc)
    Pgg_p = Pgg(kk,mu,zc,bgs8=bs8_p)
    Pgg_m = Pgg(kk,mu,zc,bgs8=bs8_m)

    return (bs8_fid/Pgg_fid)*(Pgg_p-Pgg_m)/(2*step_bs8*bs8_fid)

def dlnP_dlnDA(kk,mu,zc,step_DA=z_dep_step_default):
    DA_fid = DA(zc)
    DA_p = DA_fid*(1+step_DA)
    DA_m = DA_fid*(1-step_DA)

    Pgg_fid = Pgg(kk,mu,zc)

    k_p = kk*(DA_fid/DA_p)*np.sqrt(1+mu**2*((DA_p/DA_fid)**2 - 1)) 
    mu_p = mu*(DA_p/DA_fid)/np.sqrt(1+mu**2*((DA_p/DA_fid)**2 - 1)) 
    Pgg_p = (DA_fid/DA_p)**2 * Pgg(k_p,mu_p,zc)

    k_m = kk*(DA_fid/DA_m)*np.sqrt(1+mu**2*((DA_m/DA_fid)**2 - 1)) 
    mu_m = mu*(DA_m/DA_fid)/np.sqrt(1+mu**2*((DA_m/DA_fid)**2 - 1)) 
    Pgg_m = (DA_fid/DA_m)**2 * Pgg(k_m,mu_m,zc)

    return (DA_fid/Pgg_fid)*(Pgg_p-Pgg_m)/(2*step_DA*DA_fid)

def dlnP_dlnH(kk,mu,zc,step_H=z_dep_step_default):
    H_fid = Hz(zc)
    H_p = H_fid*(1+step_H)
    H_m = H_fid*(1-step_H)

    Pgg_fid = Pgg(kk,mu,zc)

    k_p = kk*np.sqrt(1+mu**2*((H_p/H_fid)**2 - 1)) 
    mu_p = mu*(H_p/H_fid)/np.sqrt(1+mu**2*((H_p/H_fid)**2 - 1)) 
    Pgg_p = (H_p/H_fid) * Pgg(k_p,mu_p,zc)

    k_m = kk*np.sqrt(1+mu**2*((H_m/H_fid)**2 - 1)) 
    mu_m = mu*(H_m/H_fid)/np.sqrt(1+mu**2*((H_m/H_fid)**2 - 1)) 
    Pgg_m = (H_m/H_fid) * Pgg(k_m,mu_m,zc)

    return (H_fid/Pgg_fid)*(Pgg_p-Pgg_m)/(2*step_H*H_fid)    

#bg = 1    
#print dlnP_dlnfsig8(0.01,1.0,0.8) # 0.915538125477
#print dlnP_dlnbsig8(0.01,1.0,0.8) # 1.08446187452
#print dlnP_dlnDA(0.01,1.0,0.8)    # -2.0
#print dlnP_dlnH(0.01,1.0,0.8)     # 1.40549175366

In [ ]:
# Shape parameter derivatives from scratch

# Numerical derivative wrt ns 
step_ns = shape_step*nss_fid
ns_p = nss_fid + step_ns # DERIV PLUS
ns_m = nss_fid - step_ns # DERIV MINUS    
results, rt0_ns_p = reset_cosmology(ns=ns_p); sig8_ns_p = sig8(results=results);
results, rt0_ns_m = reset_cosmology(ns=ns_m); sig8_ns_m = sig8(results=results);

# Numerical derivative wrt wb 
wb_fid = omegab_fid*pow(hubble_fid,2)
step_wb = shape_step*wb_fid
wb_p = wb_fid + step_wb # DERIV PLUS
wb_m = wb_fid - step_wb # DERIV MINUS
results, rt0_wb_p = reset_cosmology(wb=wb_p); sig8_wb_p = sig8(results=results);
results, rt0_wb_m = reset_cosmology(wb=wb_m); sig8_wb_m = sig8(results=results);

# Numerical derivative wrt wm 
wm_fid = (omegab_fid + omegac_fid)*pow(hubble_fid,2)
step_wm = shape_step*wm_fid
wm_p = wm_fid + step_wm # DERIV PLUS
wm_m = wm_fid - step_wm # DERIV MINUS
results, rt0_wm_p = reset_cosmology(wm=wm_p); sig8_wm_p = sig8(results=results);
results, rt0_wm_m = reset_cosmology(wm=wm_m); sig8_wm_m = sig8(results=results);

# Numerical derivative wrt h 
step_h = shape_step*hubble_fid
h_p = hubble_fid + step_h # DERIV PLUS
h_m = hubble_fid - step_h # DERIV MINUS
results, rt0_h_p = reset_cosmology(h=h_p); sig8_h_p = sig8(results=results);
results, rt0_h_m = reset_cosmology(h=h_m); sig8_h_m = sig8(results=results);

# RESET FIDUCIAL
_tmp, _tmp = reset_cosmology()

In [ ]:
# Get the derivatives wrt shape parameters
def dlnP_dns(kk,zc):
    return (rt0_ns_p(kk)-rt0_ns_m(kk))/(2*step_ns)/rt0_fid(kk)
        
def dlnP_dwb(kk,zc):
    return (rt0_wb_p(kk)-rt0_wb_m(kk))/(2*step_wb)/rt0_fid(kk)
    
def dlnP_dwm(kk,zc):
    return (rt0_wm_p(kk)-rt0_wm_m(kk))/(2*step_wm)/rt0_fid(kk)
    
def dlnP_dh(kk,zc):
    return (rt0_h_p(kk)-rt0_h_m(kk))/(2*step_h)/rt0_fid(kk)
    
print dlnP_dns(0.1,0.7), dlnP_dwb(0.1,0.7), dlnP_dwm(0.1,0.7), dlnP_dh(0.1,0.7)

In [ ]:
# Auxiliary bits needed for Fisher matrix calculation

def dF(kk,mu,zc):
    return (1./(8*np.pi*np.pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)  

# 2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.romb(dfun.T[i], dx=np.diff(mugrid)[0]) for i in range(kgrid.size)]
    return scipy.integrate.romb(muint, dx=np.diff(kgrid)[0]) 

## Validation

In [ ]:
ktest = 0.1
mutest = 0.5
ztest = 1.0

bg = 1.0

git = misc.GitEnv()

validation_text = '# Validating Fisher with code at commit ' + git.get_hash(7)
validation_text += "\nH(z%.1g) = %.1f"%(ztest,Hz(ztest))
validation_text += "\nD_A(z=%.1g) = %.1f"%(ztest,DA(ztest))
validation_text += "\nVsurvey(z=%.1g) = %.1f"%(ztest,Vsur(ztest))
validation_text += "\nfg(z=%.1g) = %.1f"%(ztest,fg(ztest))
validation_text += "\nsig8(z=%.1g) = %.1f"%(ztest,sig8(ztest))
validation_text += "\nPfid(z=%.1g,mu=%.1g,k=%.1g) = %.1f"%(ztest,mutest,ktest,Pkz(ktest,ztest))
validation_text += "\nPgg(z=%.1g,mu=%.1g,k=%.1g) = %.1f"%(ztest,mutest,ktest,Pgg(ktest,mutest,ztest))
validation_text += "\ndPdwb(z=%.1g,mu=%.1g,k=%.1g) = %.1f"%(ztest,mutest,ktest,dlnP_dwb(ktest,ztest))
validation_text += "\ndPdDa(z=%.1g,mu=%.1g,k=%.1g) = %.1f"%(ztest,mutest,ktest,dlnP_dlnDA(ktest,mutest,ztest))

filename = os.path.abspath(os.path.join(outpath,
                        'validation_' + git.get_hash(7) + TS + '.txt'))

with open(filename,'w') as f:
    f.write(validation_text)

In [ ]:
%%time
#####################   Fisher matrix   #####################

# Create array of zeros
Npar = len(params)
Npar_shape = len(shape_params)
s = (Npar-Npar_shape)*Nzbins + Npar_shape
Fishermat = np.zeros([s,s])

# Loop over redshift, k and mu and get all the Fisher matrix entries by calling the derivatives
kgrid = np.linspace(kmin, kmax, 513)
mugrid = np.linspace(-1., 1., 513)
K, MU = np.meshgrid(kgrid, mugrid)
for zi in range(0,Nzbins):
    zc = zlist[zi]
    bg = biaslist[zi]
    zmin = zc-Dzbin/2
    zmax = zc+Dzbin/2
    Ngal = dn3[zi]*Area*Dzbin

    for i in range(0,Npar):
        if i not in shape_params: 
            k=zi*(Npar-Npar_shape) + i
        else:
            k = s + (i - Npar)
            
        def deriv_i(kk,mu,z):
            if i==0:  return dlnP_dlnH(kk,mu,z)
            elif i==1:  return dlnP_dlnDA(kk,mu,z)
            elif i==2:  return dlnP_dlnfsig8(kk,mu,z)
            elif i==3:  return dlnP_dlnbsig8(kk,mu,z)
            elif i==4:  return dlnP_dPs(kk,mu,z)
            elif i==5:  return dlnP_dns(kk,z)
            elif i==6:  return dlnP_dwb(kk,z)
            elif i==7:  return dlnP_dwm(kk,z)
            elif i==8:  return dlnP_dh(kk,z)
            else: print "Error: index out of range"
        
        for  j in range(0,Npar):
            if j not in shape_params:
                l=zi*(Npar-Npar_shape) + j
            else:
                l = s + (j - Npar)
                       
            if j>=i:
                def deriv_j(kk,mu,z):
                    if j==0:  return dlnP_dlnH(kk,mu,z)
                    elif j==1:  return dlnP_dlnDA(kk,mu,z)
                    elif j==2:  return dlnP_dlnfsig8(kk,mu,z)
                    elif j==3:  return dlnP_dlnbsig8(kk,mu,z)
                    elif j==4:  return dlnP_dPs(kk,mu,z)
                    elif j==5:  return dlnP_dns(kk,z)
                    elif j==6:  return dlnP_dwb(kk,z)
                    elif j==7:  return dlnP_dwm(kk,z)
                    elif j==8:  return dlnP_dh(kk,z)
                    else: print "Error: index out of range" 
                               
            if l>=k: 
                Fishermat[k][l] += integrate2D(dF(K,MU,zc),kgrid,mugrid)
            else: 
                Fishermat[k,l] = Fishermat[l,k]                 

In [ ]:
# Save the resulting Fisher matrix into a file
header = 'lnH_0.7 lnDa_0.7 lnfs8_0.7 lnbs8_0.7 Ps_0.7 ' +\
         'lnH_0.8 lnDa_0.8 lnfs8_0.8 lnbs8_0.8 Ps_0.8 ' +\
         'lnH_0.9 lnDa_0.9 lnfs8_0.9 lnbs8_0.9 Ps_0.9 ' +\
         'lnH_1.0 lnDa_1.0 lnfs8_1.0 lnbs8_1.0 Ps_1.0 ' +\
         'lnH_1.1 lnDa_1.1 lnfs8_1.1 lnbs8_1.1 Ps_1.1 ' +\
         'lnH_1.2 lnDa_1.2 lnfs8_1.2 lnbs8_1.2 Ps_1.2 ' +\
         'lnH_1.3 lnDa_1.3 lnfs8_1.3 lnbs8_1.3 Ps_1.3 ' +\
         'lnH_1.4 lnDa_1.4 lnfs8_1.4 lnbs8_1.4 Ps_1.4 ' +\
         'lnH_1.5 lnDa_1.5 lnfs8_1.5 lnbs8_1.5 Ps_1.5 ' +\
         'lnH_1.6 lnDa_1.6 lnfs8_1.6 lnbs8_1.6 Ps_1.6 ' +\
         'lnH_1.7 lnDa_1.7 lnfs8_1.7 lnbs8_1.7 Ps_1.7 ' +\
         'lnH_1.8 lnDa_1.8 lnfs8_1.8 lnbs8_1.8 Ps_1.8 ' +\
         'lnH_1.9 lnDa_1.9 lnfs8_1.9 lnbs8_1.9 Ps_1.9 ' +\
         'lnH_2.0 lnDa_2.0 lnfs8_2.0 lnbs8_2.0 Ps_2.0 ' +\
         'ns wb wm h'

# Generate an output file name
newfile = os.path.abspath(os.path.join(outpath,'alkistis-case5' + '_' + interp_type))
    
# Save file
np.savetxt(newfile+TS+'.txt', Fishermat, header=header)

# Print for testing purposes
print np.sqrt(np.diag(linalg.inv(Fishermat)))